# Magliette AzIM: ordini, prezzi e consegne

L'**Azienda degli Ingegneri Matematici** ha bisogno del tuo aiuto: uno stagista chiamato F.B. ha eliminato il database degli ordini delle stupende **magliette ufficiali AzIM**! Tutto quello che rimane sono 420 frammenti di email con le richieste dei clienti, spetta a te recuperarle e determinare che ordini accettare!


0. Nella cartella data trovi tutti i file di testo con i frammenti di email, sono tutti diversi! Contengono la **taglia** delle magliette, la **quantità** di magliette ordinate, la **posizione** in coordinate GCS della consegna, il **nome** del cliente e il **prezzo** a cui vorrebbe acquistare il singolo capo


1. [1pt] Registra gli ordini in una **lista di oggetti** "Order"


2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)


3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


4. [FACOLTATIVO per lode] Considera la curvatura terrestre per il calcolo della distanza per le consegne (riscrivi la funzione "Distance")


Per qualsiasi domanda contattare Jean Paul G. Baroni +393315969286

In [63]:
# 1. Registra gli ordini in una lista di classi Order

GCS_Piazza_Leo = [45.478104, 9.227040] # Coordinate di Piazza Leonardo da Vinci, Milano
orders = [] # Lista di oggetti che descrivono gli ordini ricevuti
production_cost = 6 # Costo fisso di produzione della maglietta
num_orders = 420 # Numero di ordini nella cartella "data"

# Classe che descrive ciascun ordine ricevuto
class Order:
    
    # Costruttore della classe (prende in input il frammento di email salvato nei .txt)
    def __init__(self, email_fragment):
        
        # Separa le righe del frammento e le salva nella lista lines
        lines = email_fragment.split("\n")
        
        # Salva la taglia della maglietta
        self.size = lines[1].split(" ")[-1]
        
        # Salva la quantità di magliette richieste
        self.quantity = int(lines[1].split(" ")[-5])
        
        # Salva la posizione di consegna dell'ordine come due elementi float di una lista
        self.GCS = [float(lines[2].split(" ")[-1].split(",")[0][1:]),float(lines[2].split(" ")[-1].split(",")[1][:-1])]
        
        # Salva il nome del cliente
        self.customer = lines[5]
        
        # Salva il prezzo richiesto dal cliente (bid price)
        self.bid_price = float(lines[3].split(" ")[-1][:-1])
    
    # Calcola il pagamento che questo cliente effettuerebbe dato un prezzo di vendita
    def Payment(self, ask_price):
        result = 0
        if ask_price <= self.bid_price:
            result = ask_price*self.quantity
        return result
    
    # Calcola il profitto (ricavi - costi) per un ordine a questo cliente dato un prezzo di vendita
    def Profit(self, ask_price, cost_per_km = 0):
        result = 0
        total_cost = production_cost + cost_per_km * self.Distance()
        pay = self.Payment(ask_price)
        if pay and ask_price > total_cost:
            result = pay - total_cost*self.quantity
        return result
    
    # Calcola una distanza approssimata in km tra piazza Leonardo da Vinci (punto di partenza delle consegne) e la destinazione
    def Distance(self, starting_GCS = GCS_Piazza_Leo):
        return (((starting_GCS[0] - self.GCS[0])*111)**2+((starting_GCS[1] - self.GCS[1])*78)**2)**0.5
    
    # Stampa i dettagli dell'oggetto
    def Details(self):
        print(self.customer,"ha ordinato",self.quantity,"magliette taglia",self.size,"a "+str(self.bid_price)+"€ all'indirizzo",self.GCS)
    
# Crea gli oggetti Order e li registra nella lista orders
for i in range(num_orders):
    f = open("data/"+str(i+1)+".txt", "r")
    orders.append(Order(f.read()))
    f.close()

In [65]:
# 2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = [sum([ord2.Profit(ord1.bid_price) for ord2 in orders]) for ord1 in orders]
#_ = [print(pp) for pp in possible_profits]

# Determina il profitto maggiore
max_profit = max(possible_profits)

# Determina il prezzo migliore
best_price = [orders[i].bid_price for i in range(len(possible_profits)) if possible_profits[i] == max_profit][0]
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price) > 0]
print("Si eseguiranno", len(orders_to_go),"ordini")

Il miglior prezzo è €13.22
Si eseguiranno 195 ordini


In [64]:
# 3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.

cost_less_100 = 0.10
cost_at_least_100 = 0.05

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = [sum([ord2.Profit(ord1.bid_price,cost_at_least_100+(cost_less_100-cost_at_least_100)*int(ord1.quantity<100)) for ord2 in orders]) for ord1 in orders]
#_ = [print(pp) for pp in possible_profits]

# Determina il profitto maggiore
max_profit = max(possible_profits)

# Determina il prezzo migliore
best_price = [orders[i].bid_price for i in range(len(possible_profits)) if possible_profits[i] == max_profit][0]
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price,cost_at_least_100+(cost_less_100-cost_at_least_100)*int(ord1.quantity<100)) > 0]
print("Si eseguiranno", len(orders_to_go),"ordini")

def callProfit(ord1):
    return ord1.Profit(best_price,cost_at_least_100+(cost_less_100-cost_at_least_100)*int(ord1.quantity<100))

orders_to_go.sort(reverse = True, key = callProfit)
_ = [otg.Details() for otg in orders_to_go]

Il miglior prezzo è €16.47
Si eseguiranno 79 ordini
Filippo Fedeli Tentazioni ha ordinato 363 magliette taglia XXL a 19.2€ all'indirizzo [45.318257, 9.186061]
Gianmarco Fedeli Grissini ha ordinato 441 magliette taglia M a 16.77€ all'indirizzo [45.196751, 9.768684]
Davide Baroni Frumento ha ordinato 431 magliette taglia S a 19.62€ all'indirizzo [45.755771, 8.645121]
Carlo Augusto Fedeli Mascherine ha ordinato 357 magliette taglia XS a 17.5€ all'indirizzo [45.250094, 8.771933]
Maurizio Baroni Azienda Agricola ha ordinato 360 magliette taglia M a 17.72€ all'indirizzo [45.069648, 9.03768]
Gianmarco Bressan Company ha ordinato 491 magliette taglia XS a 19.79€ all'indirizzo [45.661732, 10.428504]
Valentina Bressan SPA ha ordinato 419 magliette taglia XL a 17.2€ all'indirizzo [45.938525, 8.407709]
JP Carrara Servizi Finanziari ha ordinato 486 magliette taglia XS a 19.87€ all'indirizzo [46.177794, 8.410633]
Angelica Bressan Strumenti Musicali ha ordinato 428 magliette taglia XL a 18.18€ all'in